In [ ]:
import os
import json
import gzip
import pandas as pd
import numpy as np

In [ ]:
path = "C:/Workspace/bookstore/" # insert the absolute path to your folder


filenames = os.listdir(path)

lines_dict = {}

for file in filenames:
    filepath = path+file
    lines_dict[file] = []
    # start reading the decompressed file's lines one by one
    # gzip takes care of the decompression
    for line in gzip.open(filepath, 'r'):
        # append each line separately into the lines list
        # this is done because json.loads can't process more
        # than 1 record at a time, see here for more:
        # https://stackoverflow.com/questions/48140858/json-decoder-jsondecodeerror-extra-data-line-2-column-1-char-190
        lines_dict[file].append(json.loads(line.decode('utf-8')))

In [ ]:
# At this point we have a lines_dict object, which has 3 keys: 1 for each loaded file
# You can view the keys as follows:
print(lines_dict.keys())

In [ ]:
# For each key, the dictionary produces a list of json rows.
# Each json row corresponds to one line for the selected file.
# The code below simply prints the number of rows (i.e. number of json lines)
# for each dictionary key
#for idx, value in enumerate(lines_dict.values()):
    #print(f"The No. {idx+1} dictionary entry corresponds to a file with {len(value)} json rows.")

In [ ]:
# extract the data for the books
isbns, titles, publishers, publication_year, book_id, authors = [], [], [], [], [], []
for line in lines_dict['goodreads_books_children.json.gz']:
    isbns.append(line['isbn'])
    titles.append(line['title'])
    publishers.append(line['publisher'])
    publication_year.append(line['publication_year'])
    book_id.append(line['book_id'])
    authors.append(line['authors'])
    

# keep only the first 100 rows for isbns that are not null
result = pd.DataFrame({'isbn' : isbns, 'book_id': book_id, 'title' : titles, 'authors': authors, 'publisher': publishers, 'publication_year': publication_year})
result[~result['isbn'].str.contains('')]
result.replace('', np.nan, inplace = True)
df_1 = result.dropna()
df1 = df_1.loc[0:150]
df = df1.drop([42,46,97,104,135]) # extract data with non-english characters


In [ ]:
# store the data that will populate the tables BOOKS_18 & PUBLISHERS_18
books_id = df["book_id"].tolist()
#print(books_id)
books_isbn = df["isbn"].tolist()
#print(books_isbn)
books_title = df["title"].tolist()

publisher = df["publisher"].tolist()

# iterate over each string in the publishers list replacing any single quote characters with two single quote characters
publisher = [name.replace("\'", "\'\'" ) for name in publisher]

publication_year= df["publication_year"].tolist()

# select publisher column, turn to df and drop the duplicates
publisher_column = df['publisher']
publisher2 = pd.DataFrame(publisher_column)
publ = publisher2.drop_duplicates()

publ2 = publ["publisher"].tolist() #turn back to list

# iterate over each string in the publishers list replacing any single quote characters with two single quote characters
publ2  = [publisher.replace("\'", "\'\'" ) for publisher in publ2] 

In [ ]:
# extract and store data from the authors column. The data resides in lists of dictionairies
authors_list = df["authors"].tolist()


author_id_values = []
role_values = []

# iterate over the sub-lists in the list
for sub_list in authors_list:
  # extract the dictionary from the sub-list
 for d in sub_list:
 
  
  # extract the values of the "author_id" and "role" keys
  author_id_value = d["author_id"]
  role_value = d["role"]
  
  # append the values to the lists
  author_id_values.append(author_id_value)
  role_values.append(role_value)

# store id's and roles as separate dataframe
  df_3= pd.DataFrame({'author_id':author_id_values, 'role':role_values})


In [ ]:
# extract the data for the authors
author_id, name = [], []
for line in lines_dict['goodreads_book_authors.json.gz']:
    author_id.append(line['author_id'])
    name.append(line['name'])
    
    

result_2 = pd.DataFrame({'author_id' : author_id, 'name' : name})

In [ ]:
# keep only the id's of relevant authors that contributed in the fist 100 books
df_2= result_2.loc[result_2['author_id'].isin(author_id_values)]

In [ ]:
#  create a dataframe to store the data that will populate the table AUTHORS_18
df_merged = df_2.merge(df_3, on= 'author_id')
df_authors = df_merged.drop_duplicates() #the merge.() method creates duplicates so we need to drop them

# store each column as a list of strings
author_id = df_authors["author_id"].tolist()
author_name = df_authors["name"].tolist()
role = df_authors["role"].tolist()

In [ ]:
# extract the data to populate the REVIEWS_18 table
books, ratings, review_id, timestamp, nickname, review_txt = [], [], [], [], [], []
for line in lines_dict['goodreads_reviews_children.json.gz']:
    ratings.append(line['rating'])
    books.append(line['book_id'])
    review_id.append(line['review_id'])
    timestamp.append(line['date_added'])
    nickname.append(line['user_id'])
    review_txt.append(line['review_text'])
    


result = pd.DataFrame({'book_id' : books, 'ratings' : ratings, 'review_id': review_id, 'timestamp': timestamp, 'nickname': nickname, 'review': review_txt})

In [ ]:
# keep only the relevant reviews for our 100 books
df_4= result.loc[result['book_id'].isin(books_id)]

# create a dataframe with books' id's and isbns only
books_id_df = pd.DataFrame({'book_id':books_id, 'isbn':books_isbn})

# merge the dataframes to create the final edition of the reviews dataframe
df_5 = df_4.merge(books_id_df, on = 'book_id')
df_reviews = df_5.drop_duplicates() #just in case there exist duplicates

In [ ]:
# store the data that will populate REVIEWS_18 table
rev_id = df_reviews["review_id"].tolist()
rev_isbn = df_reviews["isbn"].tolist()
nickname = df_reviews["nickname"].tolist()
rev_rating = df_reviews["ratings"].tolist()
rev_timestamp = df_reviews["timestamp"].tolist()
rev_text = df_reviews["review"].tolist()
rev_text  = [review.replace("\'", "\'\'" ) for review in rev_text] # use the .replace method to escape the apostroph

In [ ]:
# write SQL statements for populating the relations
with open ("data-18.sql", 'w') as f:
    f.write("INSERT INTO BOOKS_18(isbn,title,publisher,publication_year)\nVALUES\n")
    for idx, (isbn, title, publisher, publication_year) in enumerate (zip(books_isbn, books_title, publisher, publication_year)):
        if idx !=len(books_isbn)-1:
            f.write(f"{(isbn, title, publisher, publication_year)},\n")
        else:
            f.write(f"{(isbn, title, publisher, publication_year)};")
    f.write("INSERT INTO PUBLISHERS_18(publisher)\nVALUES\n")
    for idx, (publisher) in enumerate (publ2):
        if idx !=len(publ2)-1:
            f.write(f"{(publisher)},\n")
        else:
            f.write(f"{(publisher)};")
    f.write("INSERT INTO AUTHORS_18(id, name, role)\nVALUES\n")
    for idx, (id, name, role) in enumerate (zip(author_id, author_name, role)):
        if idx !=len(author_id)-1:
            f.write(f"{(id, name, role)},\n")
        else:
            f.write(f"{(id, name, role)};")
    f.write("INSERT INTO BOOKS_AUTHORS_18(isbn, id)\nVALUES\n")
    for idx, (isbn, id) in enumerate (zip(books_isbn, author_id_values)):
        if idx !=len(books_isbn)-1:
            f.write(f"{(isbn, id)},\n")
        else:
            f.write(f"{(isbn, id)};")
    f.write("INSERT INTO REVIEWS_18(review_id, isbn, nickname, ratings, timestamp, review)\nVALUES\n")
    for idx, (review_id, isbn, nickname, ratings, timestamp, review) in enumerate (zip(rev_id, rev_isbn, nickname, rev_rating, rev_timestamp, rev_text)):
        if idx !=len(rev_id)-1:
            f.write(f"{(review_id, isbn, nickname, ratings, timestamp, review)},\n")
        else:
            f.write(f"{(review_id, isbn, nickname, ratings, timestamp, review)};")